In [5]:
"""
DO NOT RUN

'Data Cleaning' is only used to trim the original datasets
"""
#
# COMBINE WITH DATA PREPARATION LATER; DON'T THINK SPLITTING THEM IS NECESSARY
#

import pandas as pd
import math

In [7]:
#creating a list of NYC zip codes and appending two extra zip codes to the list
zips_list = pd.read_csv('data/ZipCodes.txt', names=['Zip Code', 'Borough'])
ZIPLIST = list(zips_list['Zip Code'])
new_zips = [10065, 10075]
ZIPLIST = ZIPLIST + new_zips
#CHANGES: Minor optimizations, changed file reading to pandas

In [9]:
#Reading the Income Data and cleaning the header information
medianIncome21 = pd.read_csv('data/NEWDATA_MedianInc2021.csv')
tempCols = []
for header in medianIncome21.columns.values:
    tempStr = header.replace('ZCTA5 ', '')
    tempStr = tempStr.replace('!!', ' ')
    tempStr = tempStr.replace('ï»¿', '')
    tempStr = tempStr.replace('\"', '')
    tempCols.append(tempStr)
medianIncome21.columns = tempCols

medianIncome22 = pd.read_csv('data/NEWDATA_MedianInc2022.csv')
tempCols = []
for header in medianIncome22.columns.values:
    tempStr = header.replace('ZCTA5 ', '')
    tempStr = tempStr.replace('!!', ' ')
    tempStr = tempStr.replace('ï»¿', '')
    tempStr = tempStr.replace('\"', '')
    tempCols.append(tempStr)
medianIncome22.columns = tempCols
#CHANGES: changed file reading to pandas

In [11]:
#Dropping columns with zipcodes not relevant to project
first = True
for col in medianIncome21.columns.values:
    if first:
        first = False
        continue
    else:
        if int(col[0:5]) not in ZIPLIST:
            medianIncome21.drop(col, axis=1, inplace=True)

first = True
for col in medianIncome22.columns.values:
    if first:
        first = False
        continue
    else:
        if int(col[0:5]) not in ZIPLIST:
            medianIncome22.drop(col, axis=1, inplace=True)
            
#Drop rows with no data or no relevance
medianIncome21.drop([0,2,5,7,12,17,27,32,33,34,39], axis=0, inplace=True)
medianIncome22.drop([0,2,5,7,12,17,27,32,33,34,39], axis=0, inplace=True)
print(medianIncome21.columns)
#CHANGES: Minor optimizations

Index(['Label (Grouping)', '10001 Number Estimate',
       '10001 Number Margin of Error', '10001 Percent Distribution Estimate',
       '10001 Percent Distribution Margin of Error',
       '10001 Median income (dollars) Estimate',
       '10001 Median income (dollars) Margin of Error',
       '10002 Number Estimate', '10002 Number Margin of Error',
       '10002 Percent Distribution Estimate',
       ...
       '11694 Percent Distribution Estimate',
       '11694 Percent Distribution Margin of Error',
       '11694 Median income (dollars) Estimate',
       '11694 Median income (dollars) Margin of Error',
       '11697 Number Estimate', '11697 Number Margin of Error',
       '11697 Percent Distribution Estimate',
       '11697 Percent Distribution Margin of Error',
       '11697 Median income (dollars) Estimate',
       '11697 Median income (dollars) Margin of Error'],
      dtype='object', length=1243)


In [13]:
#Write cleaned data to new csv
medianIncome21.to_csv('data/NEW_21MedianInc.csv', index=False)
medianIncome22.to_csv('data/NEW_22MedianInc.csv', index=False)
#CHANGES: file writing done with pandas

In [15]:
STOPPOINT = .5 #Percent of missing data at which to ignore the row
medianRentSE = pd.read_csv('data/medianAskingRentStreetEasy.csv')
dropCols = medianRentSE.columns[3:63] #Remove columns containing data from 7+ years ago (before 2015-01)
medianRentSE.drop(dropCols, axis=1, inplace=True)
dropCols = medianRentSE.columns[-4:] #Remove 2023 columns
medianRentSE.drop(dropCols, axis=1, inplace=True)
medianRentSE.drop([0,1,2,3,4], axis=0, inplace=True)
print(medianRentSE)
#CHANGES: Minor optimizations and file reading through pandas

              areaName    Borough      areaType  2015-01  2015-02  2015-03  \
5              Astoria     Queens  neighborhood   2200.0   2200.0   2200.0   
6           Auburndale     Queens  neighborhood   1995.0      NaN   2300.0   
7           Bath Beach   Brooklyn  neighborhood   1725.0   1775.0   1775.0   
8    Battery Park City  Manhattan  neighborhood   3800.0   3950.0   4025.0   
9            Bay Ridge   Brooklyn  neighborhood   1700.0   1700.0   1749.0   
..                 ...        ...           ...      ...      ...      ...   
193    Windsor Terrace   Brooklyn  neighborhood   2700.0   2700.0   2637.0   
194          Woodhaven     Queens  neighborhood   1700.0   1700.0   1700.0   
195           Woodlawn      Bronx  neighborhood      NaN      NaN      NaN   
196           Woodside     Queens  neighborhood   1925.0   2150.0   1900.0   
197          Woodstock      Bronx  neighborhood      NaN      NaN      NaN   

     2015-04  2015-05  2015-06  2015-07  ...  2022-03  2022-04 

In [31]:
#Deleting data from over 7 years ago from medianAskingRentStreetEasy.csv, deleting neighborhoods with over
#STOPPOINT missing data points, removing extraneous data, and writing cleaned data to newMedianAskingRent.csv
first = True
newRentData = {}
cols = medianRentSE.columns[1:]
for line in medianRentSE.values:
    failcount = 0
    if first:
        DATA_LEN = len(line)
        first = False
    if line[2] == 'submarket':
        continue
    for point in line:
        if isinstance(point, str):
            continue
        if math.isnan(point):
            failcount += 1
    if failcount > (DATA_LEN * STOPPOINT): #Ignores lines that are missing more than STOPPOINT% of data
        continue
    newRentData[line[0]] = line[1:]
    
tempDF = pd.DataFrame.from_dict(newRentData).transpose()
tempDF.to_csv('data/NEW_MEDIAN_RENT.csv', header=cols)
#CHANGES: data writing done with pandas, code cleaned and optimized, STOPPOINT changed to accomodate for future change*
#Removed less yearly data (include last 7 years instead of 5 years)
# *future change is making an ML algorithm to fill in missing datapoint